In [1]:
# ================================================================
#  generate_ccw_data.py
#  --------------------
#  10 000-point sweep with +1 intrinsic fibre spin (counter-clockwise).
#  Output:  ode_steady_analysis_ccw.xlsx  saved **one directory up**.
# ================================================================

import numpy as np, pandas as pd, itertools, sys, os
from math import sin, cos, pi
from scipy.integrate import solve_ivp
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

# ---------- numerical setup -------------------------------------
T_MAX, DT = 100.0, 0.001
TIME_EVAL = np.arange(0.0, T_MAX + DT, DT)
WIN_SEC   = 10.0
WIN       = int(WIN_SEC / DT)
PLAT_TOL  = 5e-4
MATCH_TOL = 1e-2

# ---------- 10×10×10×10 grid (b = 1) ----------------------------
delta0_vals = np.linspace(-pi,  pi, 10)
p1_vals     = np.linspace(0.0, 2.0, 10)
p2_vals     = np.linspace(0.0, 2.0, 10)
a_vals      = np.linspace(1.0, 5.0, 10)
GRID = list(itertools.product(delta0_vals, p1_vals, p2_vals, a_vals))

# ---------- CCW ODE (sign = +1) ---------------------------------
def ode_ccw(t, y, p1, p2, a):
    θ, ψ = y
    b = 1.0
    s2   = sin(2*(ψ-θ))
    s, c = sin(ψ-θ), cos(ψ-θ)
    denom = 2*((a*s)**2 + (b*c)**2)**1.5
    num   = (a*b)*(a**2 - b**2)*s2
    dθ_dt = +1.0 - p1*num/denom      # ← sign flipped
    dψ_dt = -p2*s2
    return [dθ_dt, dψ_dt]

def f_delta_ccw(d, p1, p2, a):
    b = 1.0
    s2, s, c = np.sin(2*d), np.sin(d), np.cos(d)
    denom = 2*((a*s)**2 + (b*c)**2)**1.5
    dθ = +1.0 - p1*((a*b)*(a**2-b**2)*s2)/denom
    dψ = -p2*s2
    return dψ - dθ

# ---------- plateau finder --------------------------------------
def first_plateau(arr):
    for i in range(len(arr) - WIN):
        w = arr[i:i+WIN]
        if np.max(np.abs(w - w[0])) < PLAT_TOL:
            return w[0], i
    return None, None

# ---------- worker ----------------------------------------------
def run(params):
    d0, p1, p2, a = params
    θ0, ψ0 = pi/3, pi/3 + d0
    sol = solve_ivp(lambda t,y: ode_ccw(t,y,p1,p2,a),
                    (0.0, T_MAX), [θ0, ψ0],
                    t_eval=TIME_EVAL,
                    rtol=1e-6, atol=1e-9)
    if not sol.success:
        return None

    δ = sol.y[1] - sol.y[0]
    plateau_val, idx = first_plateau(δ)

    scan = np.linspace(d0 - pi, d0 + pi, 2000)
    fvals = f_delta_ccw(scan, p1, p2, a)
    zeros = [(scan[i]+scan[i+1])/2
             for i in range(len(fvals)-1) if fvals[i]*fvals[i+1] < 0]

    has_zero = bool(zeros)
    nearest  = min(zeros, key=lambda z: abs(z-d0)) if has_zero else None

    if has_zero:
        if plateau_val is not None:
            mtype = "(1)" if abs(plateau_val-nearest) < MATCH_TOL else "(2)"
        else:
            mtype = "(3)"
    else:
        mtype = "(4)" if plateau_val is None else "(5)"

    return dict(delta0=d0, p1=p1, p2=p2, a=a,
                steady_value=plateau_val,
                match_type=mtype)

# ---------- main -------------------------------------------------
def main():
    with Pool(cpu_count()) as pool:
        rows = [r for r in tqdm(pool.imap_unordered(run, GRID),
                                total=len(GRID),
                                desc="CCW sweep")
                if r is not None]

    df = pd.DataFrame(rows)
    out_path = os.path.join(os.path.dirname(os.getcwd()),
                            "ode_steady_analysis_ccw.xlsx")
    df.to_excel(out_path, index=False)
    print(f"\nSaved → {out_path}")

if __name__ == "__main__":
    main()


CCW sweep: 100%|██████████| 10000/10000 [04:29<00:00, 37.10it/s]



Saved → /scratch/jc11305/cell_research/single cell/dynamics study/GitHub upload/ode_steady_analysis_ccw.xlsx


In [2]:
import pandas as pd
import os

# Path: one directory up, matching the save location in data_for_CCW_system.py
ccw_path = os.path.join("..", "ode_steady_analysis_ccw.xlsx")

df_ccw = pd.read_excel(ccw_path)
cat_counts = df_ccw["match_type"].value_counts().sort_index()

total      = len(df_ccw)
satisfied  = cat_counts.get("(1)", 0) + cat_counts.get("(4)", 0)
prop_sat   = 100 * satisfied / total

print("Counter-clockwise (CCW) — match_type breakdown")
print(cat_counts.to_string())
print(f"\nTotal rows        : {total}")
print(f"Satisfy C1 (1)+(4): {satisfied}  ({prop_sat:.2f}%)")


Counter-clockwise (CCW) — match_type breakdown
match_type
(1)    3619
(2)    3941
(4)    2440

Total rows        : 10000
Satisfy C1 (1)+(4): 6059  (60.59%)
